In [1]:
import os
import glob
import shutil
import json
import numpy as np
from PIL import Image
from tqdm import tqdm_notebook as tqdm

In [2]:
data_dir = '../data/noaug/'
out_dir = data_dir
images_path = os.path.join(data_dir, 'images')
annot_path = data_dir + os.path.join(data_dir, 'labels')
json_name = 'coco_format_%s.json'

In [3]:
# separating images by video folders
images = glob.glob(images_path+'/*.png')
for image in tqdm(images):
    root = image.split('images/')[0]
    video = image.split('images/')[-1][:17]
    if not os.path.isdir(os.path.join(root, video)):
        os.makedirs(os.path.join(root, video))
    shutil.copyfile(image, os.path.join(root, video, image[-17:]))    

In [4]:
def get_img_size(datapath):
    img_pil = Image.open(datapath)
    width, height = img_pil.size
    return width, height


def get_groundtruth(img_path, annotation_folder):
    annot_path = None
    if annotation_folder is not None:
        # look for annotation file
        annot_path = _find_annot_file(img_path, annotation_folder)

    if annot_path is not None:
        frame_number = _get_frame_number(img_path)
        annotation = AnnotationImage(frame_number, annot_path)
        boxes, labels = annotation.get_bboxes_labels()

        return boxes, labels

    return [], []

In [16]:
sets = [
        'Train',
        'Test',
]


categories = [
    '__background__',
    'tire',
    ]

img_id = 0
ann_id = 0
cat_id = 1
category_dict = {
    #'__background__': 0,
    'tire': 0,
    }

for data_set in tqdm(sets):
    print(f'Starting {data_set}')
    
    ann_dict = {}
    images = []
    annotations = []
    
#     filenames = glob.glob(os.path.join(data_dir, data_set,'*.png'))
    
    for (dirpath, dirnames, filenames) in os.walk(os.path.join(data_dir, data_set)):
        if len(filenames) == 0:
            continue
    
        for filename in filenames:
            if filename.lower().endswith(('.png')):               
                image = {}
                
                img_name_short = os.path.join(os.path.split(dirpath)[-1], filename)
                img_name = os.path.join(dirpath, filename)
                    
                print(filename)
                width = get_img_size(img_name)[0]
                height = get_img_size(img_name)[1]

                image['file_name'] = img_name_short
                image['width'] = width
                image['height'] = height
                image['id'] = img_id
                
                
                with open(os.path.join(data_dir, 'labels', os.path.split(dirpath)[-1] + 
                                       '_'+os.path.split(filename)[-1].replace('.png','.txt')), 'r') as marks:
                    for mark in marks.readlines():
                        rect = mark.replace('\n','').split(' ')
                        objid,rcx,rcy,rw,rh = map(float,rect)
                        x0 = int((rcx-rw/2)*width)
                        y0 = int((rcy-rh/2)*height)
                        w = int(rw*width)
                        h = int(rh*height)

                        ann = {}
                        ann['id'] = ann_id
                        ann['image_id'] = img_id
                        ann['category_id'] = 0
                        ann['iscrowd'] = 0
                        ann['area'] = w*h
                        ann['bbox'] = (x0,y0,w,h)

                        annotations.append(ann)
                        ann_id += 1
                images.append(image)
                img_id += 1

            
    ann_dict['images'] = images
    categories = [{"id": category_dict[name], "name": name} for name in category_dict]
    ann_dict['categories'] = categories
    ann_dict['annotations'] = annotations
    print("Num categories: %s" % len(categories))
    print("Num images: %s" % len(images))
    print("Num annotations: %s" % len(annotations))
    
    with open(os.path.join(out_dir, json_name % data_set), 'w') as outfile:
        outfile.write(json.dumps(ann_dict))

Starting Train
frame_0001-00.png
frame_0001-01.png
frame_0001-02.png
frame_0001-10.png
frame_0001-11.png
frame_0001-12.png
frame_0001-20.png
frame_0001-21.png
frame_0001-22.png
frame_0001-30.png
frame_0001-31.png
frame_0001-32.png
frame_0001-40.png
frame_0001-41.png
frame_0001-42.png
frame_0096-00.png
frame_0176-22.png
frame_0176-30.png
frame_0176-31.png
frame_0176-32.png
frame_0176-40.png
frame_0176-41.png
frame_0176-42.png
frame_0202-00.png
frame_0202-01.png
frame_0202-02.png
frame_0202-10.png
frame_0202-11.png
frame_0202-12.png
frame_0202-20.png
frame_0202-21.png
frame_0202-22.png
frame_0202-30.png
frame_0202-31.png
frame_0202-32.png
frame_0202-40.png
frame_0202-41.png
frame_0202-42.png
frame_0230-00.png
frame_0230-01.png
frame_0230-02.png
frame_0230-10.png
frame_0230-11.png
frame_0230-12.png
frame_0520-10.png
frame_0520-11.png
frame_0520-12.png
frame_0520-20.png
frame_0520-21.png
frame_0520-22.png
frame_0520-30.png
frame_0520-31.png
frame_0520-32.png
frame_0520-40.png
frame_0520-41

frame_3500-32.png
frame_3500-40.png
frame_5038-01.png
frame_5038-02.png
frame_5038-10.png
frame_5038-11.png
frame_5038-12.png
frame_5038-20.png
frame_5038-21.png
frame_5038-22.png
frame_5038-30.png
frame_5038-31.png
frame_5038-32.png
frame_5038-40.png
frame_5038-41.png
frame_5038-42.png
frame_5067-00.png
frame_5067-01.png
frame_5067-02.png
frame_5067-10.png
frame_5067-12.png
frame_5067-20.png
frame_5067-21.png
frame_5067-22.png
frame_5067-30.png
frame_5067-31.png
frame_5067-32.png
frame_5067-40.png
frame_5067-41.png
frame_5067-42.png
frame_5096-00.png
frame_5096-01.png
frame_5096-02.png
frame_5096-10.png
frame_5096-11.png
frame_5096-12.png
frame_5096-20.png
frame_5096-21.png
frame_5096-22.png
frame_5096-30.png
frame_5096-31.png
frame_5096-32.png
frame_6278-01.png
frame_6278-02.png
frame_6278-10.png
frame_6278-11.png
frame_6278-12.png
frame_6278-20.png
frame_6278-21.png
frame_6278-22.png
frame_6278-30.png
frame_6278-31.png
frame_6278-32.png
frame_6278-40.png
frame_6278-41.png
frame_6278

frame_1704-40.png
frame_1704-41.png
frame_1704-42.png
frame_1728-00.png
frame_1728-01.png
frame_1728-02.png
frame_2261-10.png
frame_2261-11.png
frame_2261-12.png
frame_2261-20.png
frame_2261-21.png
frame_2261-22.png
frame_2261-30.png
frame_2261-31.png
frame_2261-32.png
frame_2261-40.png
frame_2261-41.png
frame_2261-42.png
frame_2291-00.png
frame_2291-01.png
frame_2291-02.png
frame_2291-10.png
frame_3740-01.png
frame_3740-02.png
frame_3740-10.png
frame_3740-11.png
frame_3740-12.png
frame_3740-20.png
frame_3740-21.png
frame_3740-22.png
frame_3740-30.png
frame_3740-31.png
frame_3740-32.png
frame_3740-40.png
frame_3740-41.png
frame_3740-42.png
frame_3768-00.png
frame_3768-01.png
frame_5438-20.png
frame_5438-21.png
frame_5438-22.png
frame_5438-30.png
frame_5438-31.png
frame_5438-32.png
frame_5438-40.png
frame_5438-41.png
frame_5438-42.png
frame_5468-00.png
frame_5468-01.png
frame_5468-02.png
frame_5468-10.png
frame_5468-11.png
frame_5468-12.png
frame_5468-20.png
frame_5468-21.png
frame_5468

frame_6031-01.png
frame_6031-02.png
frame_6031-10.png
frame_6031-11.png
frame_6031-12.png
frame_6031-20.png
frame_6031-21.png
frame_6031-22.png
frame_6031-30.png
frame_6031-31.png
frame_6031-32.png
frame_6031-40.png
frame_6031-41.png
frame_6031-42.png
frame_6062-00.png
frame_4412-30.png
frame_4412-31.png
frame_4412-32.png
frame_4412-40.png
frame_4412-41.png
frame_4412-42.png
frame_4563-00.png
frame_4563-01.png
frame_4563-02.png
frame_4563-10.png
frame_4563-11.png
frame_4563-12.png
frame_4563-20.png
frame_4563-21.png
frame_4563-22.png
frame_4563-30.png
frame_4563-31.png
frame_4563-32.png
frame_2018-30.png
frame_2018-31.png
frame_2018-32.png
frame_2018-40.png
frame_2018-41.png
frame_2018-42.png
frame_2048-00.png
frame_2048-01.png
frame_2048-02.png
frame_2048-10.png
frame_2048-11.png
frame_2048-12.png
frame_2048-20.png
frame_2048-21.png
frame_2048-22.png
frame_2048-30.png
frame_3202-42.png
frame_3229-00.png
frame_3229-01.png
frame_3229-02.png
frame_3229-10.png
frame_3229-11.png
frame_3229

frame_2494-12.png
frame_2494-20.png
frame_2494-21.png
frame_2494-22.png
frame_2494-30.png
frame_2494-31.png
frame_2494-32.png
frame_2494-40.png
frame_2494-41.png
frame_2494-42.png
frame_2523-00.png
frame_2523-01.png
frame_2523-02.png
frame_2523-10.png
frame_2523-11.png
frame_2523-12.png
frame_2523-20.png
frame_2523-21.png
frame_2523-22.png
frame_2523-30.png
frame_4798-00.png
frame_4798-01.png
frame_4798-02.png
frame_4798-10.png
frame_4798-11.png
frame_4798-12.png
frame_4798-20.png
frame_4798-21.png
frame_4798-22.png
frame_4798-30.png
frame_4798-31.png
frame_4798-32.png
frame_4798-40.png
frame_4798-41.png
frame_4798-42.png
frame_4827-00.png
frame_4827-01.png
frame_4827-10.png
frame_4827-11.png
frame_4827-12.png
frame_4827-20.png
frame_4827-21.png
frame_4827-22.png
frame_4827-30.png
frame_4827-31.png
frame_4827-32.png
frame_4827-40.png
frame_4827-41.png
frame_4827-42.png
frame_4857-00.png
frame_4857-01.png
frame_4857-02.png
frame_4857-10.png
frame_4857-11.png
frame_4857-12.png
frame_4857

...

frame_3693-31.png
frame_3693-32.png
frame_3693-40.png
frame_3693-41.png
frame_3693-42.png
frame_3719-00.png
frame_3719-01.png
frame_3719-02.png
frame_3719-10.png
frame_3719-11.png
frame_3719-12.png
frame_3719-20.png
frame_3719-21.png
frame_3719-22.png
frame_3719-30.png
frame_3824-32.png
frame_3824-40.png
frame_3824-41.png
frame_3824-42.png
frame_3850-00.png
frame_3850-01.png
frame_3850-02.png
frame_3850-10.png
frame_3850-11.png
frame_3850-12.png
frame_3850-20.png
frame_3850-21.png
frame_3850-22.png
frame_3850-30.png
frame_3850-31.png
frame_3850-32.png
frame_2627-22.png
frame_2627-30.png
frame_2627-31.png
frame_2627-32.png
frame_2627-40.png
frame_2627-41.png
frame_2627-42.png
frame_2825-00.png
frame_2825-01.png
frame_2825-02.png
frame_2825-10.png
frame_2825-11.png
frame_2825-12.png
frame_2825-20.png
frame_2825-21.png
frame_2825-22.png
frame_2825-30.png
frame_2825-31.png
frame_2825-32.png
frame_2825-40.png
frame_4226-41.png
frame_4226-42.png
frame_4254-00.png
frame_4254-01.png
frame_4254

frame_0751-30.png
frame_0751-31.png
frame_0751-32.png
frame_0751-40.png
frame_0751-41.png
frame_0751-42.png
frame_0805-00.png
frame_0805-01.png
frame_5764-01.png
frame_5764-02.png
frame_5764-10.png
frame_5764-11.png
frame_5764-12.png
frame_5764-20.png
frame_5764-21.png
frame_5764-22.png
frame_5764-30.png
frame_5764-31.png
frame_5764-32.png
frame_5764-40.png
frame_5764-41.png
frame_5764-42.png
frame_5808-00.png
frame_5808-01.png
frame_5808-02.png
frame_3537-32.png
frame_3537-40.png
frame_3537-41.png
frame_3537-42.png
frame_3563-00.png
frame_3563-01.png
frame_3563-02.png
frame_3563-10.png
frame_3563-11.png
frame_3563-12.png
frame_3563-20.png
frame_3563-21.png
frame_3563-22.png
frame_3563-30.png
frame_3563-31.png
frame_3563-32.png
frame_5944-10.png
frame_5944-11.png
frame_5944-12.png
frame_5944-20.png
frame_5944-21.png
frame_5944-22.png
frame_5944-30.png
frame_5944-31.png
frame_5944-32.png
frame_5944-40.png
frame_5944-41.png
frame_5944-42.png
frame_5991-00.png
frame_5991-01.png
frame_5991

frame_1621-00.png
frame_1621-01.png
frame_1621-02.png
frame_1621-10.png
frame_1621-11.png
frame_6692-40.png
frame_6692-41.png
frame_6692-42.png
frame_6739-00.png
frame_6739-01.png
frame_6739-02.png
frame_6739-10.png
frame_6739-11.png
frame_6739-12.png
frame_6739-20.png
frame_6739-21.png
frame_6739-22.png
frame_6739-30.png
frame_6739-31.png
frame_6739-32.png
frame_6739-40.png
frame_6739-41.png
frame_6739-42.png
frame_6785-00.png
frame_6785-01.png
frame_6785-02.png
frame_6785-10.png
frame_6785-11.png
frame_6785-12.png
frame_6785-20.png
frame_6785-21.png
frame_6785-22.png
frame_6785-30.png
frame_6785-31.png
frame_3219-01.png
frame_3219-02.png
frame_3219-10.png
frame_3219-11.png
frame_3219-12.png
frame_3219-20.png
frame_3219-21.png
frame_3219-22.png
frame_3219-30.png
frame_3219-31.png
frame_3219-32.png
frame_3219-40.png
frame_3219-41.png
frame_3219-42.png
frame_3247-00.png
frame_3247-01.png
frame_3247-02.png
frame_3043-01.png
frame_3043-02.png
frame_3043-10.png
frame_3043-11.png
frame_3043

frame_4942-20.png
frame_4942-21.png
frame_4942-22.png
frame_4942-30.png
frame_4942-31.png
frame_4942-32.png
frame_4942-40.png
frame_4942-41.png
frame_4942-42.png
frame_5153-00.png
frame_5153-01.png
frame_5153-02.png
frame_5153-10.png
frame_5153-11.png
frame_5898-00.png
frame_5898-01.png
frame_5898-02.png
frame_5898-10.png
frame_5898-11.png
frame_5898-12.png
frame_5898-20.png
frame_5898-21.png
frame_5898-22.png
frame_5898-30.png
frame_5898-31.png
frame_5898-32.png
frame_5898-40.png
frame_5898-41.png
frame_5898-42.png
frame_5944-00.png
frame_5944-01.png
frame_6831-42.png
frame_6877-00.png
frame_6877-01.png
frame_6877-02.png
frame_6877-10.png
frame_6877-11.png
frame_6877-12.png
frame_6877-20.png
frame_6877-21.png
frame_6877-22.png
frame_6877-30.png
frame_6877-31.png
frame_6877-32.png
frame_6877-40.png
frame_6877-41.png
frame_6877-42.png
frame_6924-00.png
frame_6924-01.png
frame_6924-02.png
frame_6924-10.png
frame_6924-11.png
frame_5586-21.png
frame_5586-22.png
frame_5586-30.png
frame_5586

frame_3329-30.png
frame_3329-31.png
frame_3329-32.png
frame_3329-40.png
frame_3329-41.png
frame_3329-42.png
frame_3355-00.png
frame_3355-01.png
frame_3355-02.png
frame_3355-10.png
frame_3355-11.png
frame_3355-12.png
frame_3355-20.png
frame_3077-42.png
frame_3107-00.png
frame_3107-01.png
frame_3107-02.png
frame_3107-10.png
frame_3107-11.png
frame_3107-12.png
frame_3107-20.png
frame_3107-21.png
frame_3107-22.png
frame_3107-30.png
frame_3107-31.png
frame_3107-32.png
frame_3107-40.png
frame_3107-41.png
frame_3107-42.png
frame_0975-12.png
frame_0975-20.png
frame_0975-21.png
frame_0975-22.png
frame_0975-30.png
frame_0975-31.png
frame_0975-32.png
frame_0975-40.png
frame_0975-41.png
frame_0975-42.png
frame_1029-00.png
frame_1029-01.png
frame_1029-02.png
frame_1029-10.png
frame_1029-11.png
frame_1029-12.png
frame_2939-01.png
frame_2939-02.png
frame_2939-10.png
frame_2939-11.png
frame_2939-12.png
frame_2939-20.png
frame_2939-21.png
frame_2939-22.png
frame_2939-30.png
frame_2939-31.png
frame_2939